## Deliverable 2. Create a Customer Travel Destinations Map.

In [5]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

# get file path
DSB_path = os.getenv('DSB_PATH')

In [13]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv(f"{DSB_path}6_WeatherPywAPI/world_weather_analysis/Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,New Norfolk,AU,2022-09-03 20:11:37,-42.7826,147.0587,42.93,81,53,2.57,broken clouds
1,1,Marsh Harbour,BS,2022-09-03 20:14:45,26.5412,-77.0636,84.36,66,100,10.98,overcast clouds
2,2,Qaanaaq,GL,2022-09-03 20:15:54,77.4840,-69.3632,38.82,96,100,9.24,light rain
3,3,Barrow,US,2022-09-03 20:15:54,71.2906,-156.7887,39.22,93,100,8.05,overcast clouds
4,4,Olafsvik,IS,2022-09-03 20:15:54,64.8945,-23.7142,49.42,72,100,6.67,overcast clouds


In [14]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [22]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Marsh Harbour,BS,2022-09-03 20:14:45,26.5412,-77.0636,84.36,66,100,10.98,overcast clouds
7,7,Georgetown,MY,2022-09-03 20:13:27,5.4112,100.3354,80.53,93,20,2.30,thunderstorm
11,11,Albany,US,2022-09-03 20:11:18,42.6001,-73.9662,81.68,58,100,1.01,overcast clouds
16,16,Butaritari,KI,2022-09-03 20:15:58,3.0707,172.7902,80.92,80,33,16.33,light rain
18,18,Samarai,PG,2022-09-03 20:15:58,-10.6167,150.6667,80.40,81,1,9.01,clear sky
22,22,Sao Joao Da Barra,BR,2022-09-03 20:13:00,-21.6403,-41.0511,77.09,80,0,18.32,clear sky
30,30,Shwebo,MM,2022-09-03 20:16:03,22.5667,95.7000,82.47,73,46,10.63,scattered clouds
39,39,Kapaa,US,2022-09-03 20:16:07,22.0752,-159.3190,82.38,76,54,7.00,broken clouds
42,42,Ribeira Grande,PT,2022-09-03 20:16:08,38.5167,-28.7000,75.60,60,20,4.61,few clouds
46,46,Fayaoue,NC,2022-09-03 20:17:09,-20.6500,166.5333,75.40,75,96,14.70,overcast clouds


In [23]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 188 entries, 1 to 688
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City_ID              188 non-null    int64  
 1   City                 188 non-null    object 
 2   Country              188 non-null    object 
 3   Date                 188 non-null    object 
 4   Lat                  188 non-null    float64
 5   Lng                  188 non-null    float64
 6   Max Temp             188 non-null    float64
 7   Humidity             188 non-null    int64  
 8   Cloudiness           188 non-null    int64  
 9   Wind Speed           188 non-null    float64
 10  Current Description  188 non-null    object 
dtypes: float64(4), int64(3), object(4)
memory usage: 17.6+ KB


In [26]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
## set the subset to City so that cities with at least some data are still included. only fully empty rows would be dropped
clean_df = preferred_cities_df.dropna(subset=['City'])

In [27]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Marsh Harbour,BS,84.36,overcast clouds,26.5412,-77.0636,
7,Georgetown,MY,80.53,thunderstorm,5.4112,100.3354,
11,Albany,US,81.68,overcast clouds,42.6001,-73.9662,
16,Butaritari,KI,80.92,light rain,3.0707,172.7902,
18,Samarai,PG,80.40,clear sky,-10.6167,150.6667,
22,Sao Joao Da Barra,BR,77.09,clear sky,-21.6403,-41.0511,
30,Shwebo,MM,82.47,scattered clouds,22.5667,95.7000,
39,Kapaa,US,82.38,broken clouds,22.0752,-159.3190,
42,Ribeira Grande,PT,75.60,few clouds,38.5167,-28.7000,
46,Fayaoue,NC,75.40,overcast clouds,-20.6500,166.5333,


In [28]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']

    params['location'] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url ="https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [40]:
# 7. Drop the rows where there is no Hotel Name.
## dropna method not used as hotel nulls came in as "" and not NA
clean_hotel_df = hotel_df[hotel_df['Hotel Name']!=""]

# confirm rows were dropped--if any hotel not found skip message above, second results should be less than first
print(len(hotel_df),len(clean_hotel_df))

188 177


In [41]:
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Marsh Harbour,BS,84.36,overcast clouds,26.5412,-77.0636,Abaco Beach Resort
7,Georgetown,MY,80.53,thunderstorm,5.4112,100.3354,Cititel Penang
16,Butaritari,KI,80.92,light rain,3.0707,172.7902,Isles Sunset Lodge
18,Samarai,PG,80.40,clear sky,-10.6167,150.6667,Nuli Sapi
22,Sao Joao Da Barra,BR,77.09,clear sky,-21.6403,-41.0511,Pousada Porto De Canoas


In [42]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [49]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt><b>Hotel Name</b></dt><dd>{Hotel Name}</dd>
<dt><b>City</b></dt><dd>{City}</dd>
<dt><b>Country</b></dt><dd>{Country}</dd>
<dt><b>Current Weather</b></dt><dd>{Current Description} and {Max Temp}°F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [54]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))